In [30]:
import sys
import geopandas
import pandas as pd
import numpy as np
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

## load GIS vector data - waldorte

In [36]:
wo_geo = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_test/joined.shp')

In [37]:
wo_geo.head()

,OBJECTID,FORSTBETRI,REVIER_NR,ABTEILUNG,UNTERABTEI,TEILFLAECH,FARBCODE,LINKID,ID,VERWALTUNG,Erstellung,Datenaktua,SHAPE_Leng,SHAPE_Area,FLUGJAHR,geometry
0,1.0,171,1,-1,0,1,00000000000,{7DDA8BDD-4E69-47D2-B7B5-D75FBCAAA0B2},1450126,529,2020/11/20 09:37:18.000,2019/01/01 00:00:00.000,195.920109,1553.41910,2018.0,"POLYGON ((616346.810 493670.595, 616342.890 49..."
1,2.0,171,1,-1,0,1,00000000000,{2CAE2042-5D26-4B61-857B-A3643351503C},1450044,529,2020/11/20 09:37:18.000,2019/01/01 00:00:00.000,1125.586021,35217.65235,2018.0,"POLYGON ((615325.040 492902.555, 615263.980 49..."
2,3.0,171,1,-1,0,1,00000000000,{62D0D4D2-FFD0-45DD-8622-5B24494B182C},1449991,529,2020/11/20 09:37:18.000,2019/01/01 00:00:00.000,946.503132,24184.69905,2018.0,"POLYGON ((608107.790 487993.105, 608108.120 48..."
3,4.0,171,1,-1,0,1,00000000000,{1AD6A0D3-C110-4E44-B7D8-DE5495E5C3EC},1450382,529,2020/11/20 09:37:18.000,2019/01/01 00:00:00.000,319.914046,5266.03840,2018.0,"POLYGON ((614628.190 492540.735, 614612.100 49..."
4,5.0,171,1,506,3,1,00000000003,{E77BB096-11C9-4939-9331-B4B305C4096A},1450402,529,2020/11/20 09:37:18.000,2019/01/01 00:00:00.000,2402.546849,11624.65550,2018.0,"POLYGON ((617139.640 493799.045, 617151.260 49..."


In [38]:
wo_geo.crs

{'init': 'epsg:31287'}

In [39]:
# filter abteilung -1
wo_geo = wo_geo.loc[wo_geo['ABTEILUNG'] != -1,:]

In [40]:
# create unique ID WO
wo_geo['WO'] = wo_geo['FORSTBETRI'].astype(str) + \
wo_geo['REVIER_NR'].astype(str).str.zfill(2) + \
wo_geo['ABTEILUNG'].astype(str).str.zfill(3) + \
wo_geo['UNTERABTEI'] + \
wo_geo['TEILFLAECH'].astype(str)

In [41]:
wo_geo['FLUGJAHR'] = wo_geo['FLUGJAHR'].fillna(0)
wo_geo['FLUGJAHR'] = wo_geo['FLUGJAHR'].astype(int)

In [42]:
wo_geo = wo_geo[wo_geo['FLUGJAHR'] != 0]

## load GIS vector data - orthophoto metadata

In [34]:
meta_geo = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_meta/flugjahr.shp')

In [10]:
meta_geo = meta_geo[['FLUGJAHR', 'geometry']]

In [35]:
meta_geo.head()

,FLUG_DAT,BOD_AUFL,SPEKTRUM,KAMERATYP,BEMERKUNG,MBL_NAME,DAT_LIEF_A,LFE_ID,LIEF_DAT,BL_NAME,FLUGJAHR,geometry
0,05.10.2018,0.2,RGBI,None,None,3621-47,23.05.2019,1619.0,17.04.2019,TIR,2018,"MULTIPOLYGON Z (((376255.203 299731.999 0.000,..."
1,08.08.2017,0.2,RGBI,None,None,4941-70,03.05.2018,1502.0,30.01.2018,OOE,2017,"MULTIPOLYGON Z (((528150.720 311746.428 0.000,..."
2,22.05.2016,0.2,RGBI,None,None,7827-24,19.06.2017,1413.0,22.12.2016,BGL,2016,"MULTIPOLYGON Z (((493732.849 274749.378 0.000,..."
3,26.08.2015,0.2,RGBI,None,None,1421-54,13.06.2016,1377.0,29.03.2016,TIR,2015,"MULTIPOLYGON Z (((224688.559 321076.871 0.000,..."
4,17.08.2013,0.0,RGBI,None,None,4318-06,27.05.2014,1126.0,02.04.2014,KTN,2013,"POLYGON Z ((401249.684 315727.538 0.000, 40000..."


### preform spatial join

In [ ]:
wo_geo = geopandas.sjoin(wo_geo, meta_geo, how='left', op='intersects')

In [ ]:
wo_geo.head()

## load SAP tax data

In [10]:
# get infromation about which Teiloperate were active in a giveb year

In [11]:
# set year
year = 2019

path_sap_info = '/home/philipp/Data/edin_diss/SAP_tax/edin_meta_data.xlsx'
sap_info = pd.read_excel(path_sap_info)

In [12]:
sap_info.head()

,FB,FR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,171,1,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
1,171,2,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
2,171,3,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
3,171,4,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
4,171,5,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356


In [13]:
# array with all TOs in a given year
tos = sap_info.loc[:,year].unique()

In [14]:
tos

array([1030, 1353, 1049, 1053, 1052, 1100, 1050, 1051, 1083, 1042, 1073,
       1092, 1311, 1342, 1151, 1194, 1086, 1043, 1144, 1197, 1055, 1303,
       1085, 1345, 1302, 1140, 1312, 1065, 1066, 1067, 1088, 1044, 1196,
       1313, 1343, 1354, 1093, 1045, 1081, 1082, 1341, 1355, 1156, 1250])

In [15]:
def get_data(tos):
    # set sap tax path directory
    path_sap_tax_dir = '/home/philipp/Data/edin_diss/SAP_tax'

    wo_sap_list = []

    for to in tos:
        # get fb
        fb = sap_info.loc[sap_info[year] == to, 'FB'].unique()[0]
        # create path to file
        path_sap_tax_file = path_sap_tax_dir + '/' + str(fb) + '/TO_' + str(to) + '.XLS'

        print(path_sap_tax_file)

        # read dat from file
        wo_sap_list.append(pd.read_csv(path_sap_tax_file, 
                                       sep='\t',
                                       encoding = "ISO-8859-1", 
                                       decimal=',', 
                                       error_bad_lines=False))
        # create unique ID WO
        #wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \
        #wo_sap['Forstrevier'].astype(str) + \
        #wo_sap['Abteilung'].astype(str) + \
        #wo_sap['Unterabteil.'] + \
        #wo_sap['Teilfl.'].astype(str)

        #wo_sap_list.append(wo_sap)

    return pd.concat(wo_sap_list, ignore_index=True)

In [16]:
wo_sap = get_data(tos)

/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (29,127,129,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/171/TO_1353.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/171/TO_1049.XLS


...

/home/philipp/Data/edin_diss/SAP_tax/179/TO_1196.XLS
/home/philipp/Data/edin_diss/SAP_tax/179/TO_1313.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (148) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/180/TO_1343.XLS
/home/philipp/Data/edin_diss/SAP_tax/180/TO_1354.XLS


/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (40,146,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/home/philipp/Data/edin_diss/SAP_tax/181/TO_1093.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1045.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1081.XLS
/home/philipp/Data/edin_diss/SAP_tax/181/TO_1082.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1341.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1355.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1156.XLS
/home/philipp/Data/edin_diss/SAP_tax/182/TO_1250.XLS


In [17]:
wo_sap['Unterabteil.'].unique()

array(['B', 'C', 'A', 'D', 'G', 'F', 'E', 'L', 'H', 'N', 'I', 'K', '5',
       '3', '4', 'M', 'O', 'J', 'R', 'S', 'P', '9', '7', '8', '6', '1',
       'U', 'T', 'Z', 'V', 'W', 'Q', 'X', 'Y', '2'], dtype=object)

In [18]:
# create unique ID WO
wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \
wo_sap['Forstrevier'].astype(str).str.zfill(2) + \
wo_sap['Abteilung'].astype(str).str.zfill(3) + \
wo_sap['Unterabteil.'] + \
wo_sap['Teilfl.'].astype(str)

In [19]:
wo_sap.head()

,Merkmalausprägung,AuswKatTyp,Teiloperats-ID,Forstbetrieb,Debitor,TO-Bezeichnung,Status,Beg. Laufzeit,Ende Laufzeit,Operat-ID,...,Zeile2,Zeile3,Zeile4,Zeile5,Zeile6,Zeile7,Zeile8,Bestockungsziel,Flächenanteil,WO
0,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,BE Im N-TL gelichtet und winddurchrissen mit ...,diese hier älter (+5 J); ein Einzel-WW; im N-T...,vergrast und leicht verkrautet,MA BZ: 5LA 5BU,NaN,NaN,NaN,NaN,0.0,17101648B1
1,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17101648B1
2,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17101648B1
3,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17101648B1
4,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17101648B1


In [20]:
# stoe

# filter data
wo_sap_stoe = wo_sap.loc[wo_sap['Best.-Schicht'] == 0, ['WO', 'Forstbetrieb', 'Forstrevier', 'Abteilung', 
                                                        'Unterabteil.', 'Teilfl.', 'Beg. Laufzeit', 'Umtriebszeit', 
                                                        'Nebengrund Art', 'Ertragssituation', 'Bewirtschaftungsform', 
                                                        'Schutzwaldkategorie', 'Seehöhe', 'Exposition', 'Neigung', 
                                                        'Standorteinheit', 'Vegetationstyp', 'Wuchsgebiet']]

wo_sap_stoe.head()

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,Bewirtschaftungsform,Schutzwaldkategorie,Seehöhe,Exposition,Neigung,Standorteinheit,Vegetationstyp,Wuchsgebiet
0,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,W,NaN,400,SO,27,87.0,BW,5.1
6,17101652C1,171,1,652,C,1,01.01.2009,120,NaN,I,W,NaN,400,SW,27,87.0,WW,5.1
12,17101736B4,171,1,736,B,4,01.01.2009,120,NaN,I,W,NaN,300,S,18,87.0,WW,5.1
21,17101658A1,171,1,658,A,1,01.01.2009,120,NaN,I,W,NaN,300,SW,18,87.0,WW,5.1
27,17101506D1,171,1,506,D,1,01.01.2009,120,NaN,I,W,NaN,300,NO,27,88.0,WW,5.1


In [21]:
# wood volume

# filter data
wo_sap_v = wo_sap.loc[wo_sap['Best.-Schicht.1'] > 0, ['WO', 'Vorrat / ha', 'Laubholzvorrat / ha', 
                                                      'Nadelholzvorrat / ha', 'Vorrat am Ort', 
                                                      'Laubholzvorrat Ort', 'Nadelholzvorrat Ort',]]

# group by WO (ID) and sum all values
wo_sap_v = wo_sap_v.groupby(['WO']).sum().reset_index()

wo_sap_v.head()

,WO,Vorrat / ha,Laubholzvorrat / ha,Nadelholzvorrat / ha,Vorrat am Ort,Laubholzvorrat Ort,Nadelholzvorrat Ort
0,17101506A0,331.2,229.68,101.52,3083.47,2138.32,945.15
1,17101506B1,236.7,208.18,28.52,295.88,260.23,35.65
2,17101506B2,13.0,13.00,0.00,7.28,7.28,0.00
3,17101506C1,175.0,175.00,0.00,428.75,428.75,0.00
4,17101506C2,89.0,72.90,16.10,263.44,215.78,47.66


In [22]:
# wood cuts

# filter data
wo_sap_ma = wo_sap.loc[wo_sap['Nutzungsnummer'] > 0, ['WO', 'Maßnahmenart', 'Massnahmengruppe', 'Angriffsfläche', 
                                                      'Nutzung LH', 'Nutzung NH', 'Nutzungssumme', 'Nutzdringlichkeit', 
                                                      'Bewpfl.', 'Zeitpunkt', 'Rückungsart', 'Schlägerungsart']]

wo_sap_ma.head()

,WO,Maßnahmenart,Massnahmengruppe,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
5,17101648B1,RM,EN,4.0,1600.0,0.0,1600.0,2,3,2,30.0,1.0
10,17101652C1,DF,VN,3.0,100.0,0.0,100.0,2,1,2,30.0,1.0
11,17101652C1,ZE,EN,0.0,30.0,20.0,50.0,3,1,2,30.0,1.0
20,17101736B4,JP,WP,3.5,0.0,0.0,0.0,1,1,1,NaN,NaN
26,17101658A1,DF,VN,4.0,200.0,0.0,200.0,2,1,2,10.0,1.0


In [23]:
# filter just VN

In [24]:
wo_sap_maf = wo_sap_ma[wo_sap_ma['Massnahmengruppe'] == 'VN']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'ZV']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'UE']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'LL']

In [25]:
wo_sap_maf['Maßnahmenart'].unique()

array(['DF', 'DE', 'ND'], dtype=object)

In [26]:
# get dublicated records
#wo_d = wo_sap_maf[wo_sap_maf.duplicated(subset = 'WO')]

In [27]:
# group by WO (ID) and sum all values
wo_sap_maf = wo_sap_maf.groupby(['WO']).sum().reset_index()

In [28]:
wo_sap_maf

,WO,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
0,17101506A0,9.3,350.0,150.0,500.0,2,1,2,30.0,1.0
1,17101506C1,0.6,30.0,0.0,30.0,1,1,2,10.0,1.0
2,17101506C2,3.2,165.0,0.0,165.0,3,2,4,45.0,5.0
3,17101506D2,2.8,120.0,0.0,120.0,2,1,2,35.0,4.0
4,17101506G1,1.9,60.0,30.0,90.0,1,1,2,35.0,4.0
...,...,...,...,...,...,...,...,...,...,...
37465,18209366L1,2.5,0.0,120.0,120.0,1,1,1,35.0,4.0
37466,18209366L2,0.1,0.0,10.0,10.0,2,1,1,35.0,4.0
37467,18209366M2,1.9,0.0,80.0,80.0,1,1,1,35.0,4.0
37468,18209367H2,0.6,0.0,20.0,20.0,2,1,1,30.0,2.0


### merge all SAP data

In [29]:
# merge SAP stoe & SAP volume
wo_sap = pd.merge(wo_sap_stoe, wo_sap_v, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

# merge
wo_sap = pd.merge(wo_sap, wo_sap_maf, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

wo_sap.head()

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,...,Nadelholzvorrat Ort,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
0,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17101652C1,171,1,652,C,1,01.01.2009,120,NaN,I,...,777.17,3.0,100.0,0.0,100.0,2.0,1.0,2.0,30.0,1.0
2,17101736B4,171,1,736,B,4,01.01.2009,120,NaN,I,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17101658A1,171,1,658,A,1,01.01.2009,120,NaN,I,...,0.00,4.0,200.0,0.0,200.0,2.0,1.0,2.0,10.0,1.0
4,17101506D1,171,1,506,D,1,01.01.2009,120,NaN,I,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
wo_sap.to_csv('/home/philipp/Data/edin_diss/SAP.csv')

In [31]:
wo_sap = pd.read_csv('/home/philipp/Data/edin_diss/SAP.csv')

## load SAP nutz

In [32]:
def get_data_sap_nutz():
    # set sap tax path directory
    path_sap_nutz_dir = '/home/philipp/Data/edin_diss/SAP_nutz'

    sap_nutz_list = []

    for fb in range(171,183):
        # create path to file
        path_sap_file = path_sap_nutz_dir + '/FB' + str(fb) + '_2007_2019.xlsx'
        print(path_sap_file)
        # read dat from file
        sap_nutz_list.append(pd.read_excel(path_sap_file))

    return pd.concat(sap_nutz_list, ignore_index=True)

In [32]:
sap_nutz = get_data_sap_nutz()

/home/philipp/Data/edin_diss/SAP_nutz/FB171_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB172_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB173_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB174_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB175_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB176_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB177_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB178_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB179_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB180_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB181_2007_2019.xlsx
/home/philipp/Data/edin_diss/SAP_nutz/FB182_2007_2019.xlsx


In [33]:
# fill nan in 'Ergebnis'
sap_nutz = sap_nutz.fillna('Ergebnis')
# filter subtotals
sap_nutz = sap_nutz[sap_nutz['planmäßig'] != 'Ergebnis']
# reset index
sap_nutz = sap_nutz.reset_index(drop=True)

In [34]:
sap_nutz

,Abmaßjahr,Waldort,Nutzungsart,planmäßig,Menge Efm
0,2007,#/171/1/710C1,Räumung,planmäßig,175.18
1,2007,#/171/1/740A0,Räumung,planmäßig,314.12
2,2007,#/171/3/146C1,Durchforstung,planmäßig,28.40
3,2007,#/171/4/421B0,Schäden andere,zufällig,25.20
4,2007,#/171/5/294G0,Schäden Wind,zufällig,8.39
...,...,...,...,...,...
340306,2019,1355/182/5/297O5,Schäden Käfer,zufällig,1.74
340307,2019,1355/182/5/297O5,Schäden Wind,zufällig,5.22
340308,2019,1355/182/5/297O6,Schäden Käfer,zufällig,1.74
340309,2019,1355/182/5/297O7,Schäden Käfer,zufällig,1.16


In [35]:
# 
ids = sap_nutz['Waldort'].str.split('/')
# clear Waldort
sap_nutz['Waldort'] = ids.str[1] + ids.str[2].str.zfill(2) + ids.str[3].str.zfill(5)

In [36]:
sap_nutz['Waldort']

0         17101710C1
1         17101740A0
2         17103146C1
3         17104421B0
4         17105294G0
             ...    
340306    18205297O5
340307    18205297O5
340308    18205297O6
340309    18205297O7
340310    18205521G1
Name: Waldort, Length: 340311, dtype: object

In [37]:
# group by WO (ID) and sum all values
sap_nutz = sap_nutz.groupby(['Abmaßjahr','Waldort']).sum().reset_index()

In [38]:
sap_nutz.columns = ['cut_year', 'WO', 'cut_volume']

In [39]:
sap_nutz

,cut_year,WO,cut_volume
0,2007,17101506C1,7.800
1,2007,17101507B1,4.550
2,2007,17101508D1,736.480
3,2007,17101510A2,8.450
4,2007,17101510C1,477.180
...,...,...,...
259961,2019,18209366H1,29.400
259962,2019,18209366H2,2.950
259963,2019,18209367F1,27.103
259964,2019,18209367K1,37.585


In [42]:
# merge
wo_sap = pd.merge(wo_sap, sap_nutz, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

wo_sap.head()

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,...,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart,cut_year,cut_volume
0,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007,209.23
1,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008,631.13
2,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009,230.76
3,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,413.43
4,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,342.05


In [47]:
wo_sap['FE_year'] = wo_sap['Beg. Laufzeit'].str[-4:].astype(int)

In [48]:
wo_sap

,Unnamed: 0,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,...,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart,FE_year
0,0,17101648B1,171,1,648,B,1,01.01.2009,120,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
1,1,17101652C1,171,1,652,C,1,01.01.2009,120,NaN,...,3.0,100.0,0.0,100.0,2.0,1.0,2.0,30.0,1.0,2009
2,2,17101736B4,171,1,736,B,4,01.01.2009,120,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
3,3,17101658A1,171,1,658,A,1,01.01.2009,120,NaN,...,4.0,200.0,0.0,200.0,2.0,1.0,2.0,10.0,1.0,2009
4,4,17101506D1,171,1,506,D,1,01.01.2009,120,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231153,231153,18208062B5,182,8,62,B,5,01.01.2016,140,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
231154,231154,18208022A3,182,8,22,A,3,01.01.2016,140,NaN,...,4.0,0.0,150.0,150.0,1.0,1.0,2.0,10.0,1.0,2016
231155,231155,18208020B2,182,8,20,B,2,01.01.2016,140,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
231156,231156,18209324C4,182,9,324,C,4,01.01.2016,140,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016


In [49]:
wo = wo_geo.merge(wo_sap, on='WO')

In [51]:
wo.columns

Index(['OBJECTID', 'FORSTBETRI', 'REVIER_NR', 'ABTEILUNG', 'UNTERABTEI',
       'TEILFLAECH', 'FARBCODE', 'LINKID', 'ID', 'VERWALTUNG', 'Erstellung',
       'Datenaktua', 'SHAPE_Leng', 'SHAPE_Area', 'FLUGJAHR', 'geometry', 'WO',
       'Unnamed: 0', 'Forstbetrieb', 'Forstrevier', 'Abteilung',
       'Unterabteil.', 'Teilfl.', 'Beg. Laufzeit', 'Umtriebszeit',
       'Nebengrund Art', 'Ertragssituation', 'Bewirtschaftungsform',
       'Schutzwaldkategorie', 'Seehöhe', 'Exposition', 'Neigung',
       'Standorteinheit', 'Vegetationstyp', 'Wuchsgebiet', 'Vorrat / ha',
       'Laubholzvorrat / ha', 'Nadelholzvorrat / ha', 'Vorrat am Ort',
       'Laubholzvorrat Ort', 'Nadelholzvorrat Ort', 'Angriffsfläche',
       'Nutzung LH', 'Nutzung NH', 'Nutzungssumme', 'Nutzdringlichkeit',
       'Bewpfl.', 'Zeitpunkt', 'Rückungsart', 'Schlägerungsart', 'FE_year'],
      dtype='object')

In [52]:
wo[['FE_year', 'FLUGJAHR']]

,FE_year,FLUGJAHR
0,2009,2018
1,2009,2018
2,2009,2018
3,2009,2018
4,2009,2018
...,...,...
370528,2015,2013
370529,2015,2013
370530,2015,2013
370531,2015,2013
